In [1]:
# Download CBDB sqlite
# !git clone https://github.com/cbdb-project/cbdb_sqlite.git
import requests
import os
url = "https://hf-mirror.com/datasets/cbdb/cbdb-sqlite/resolve/main/latest.7z?download=true"
# url = "https://huggingface.co/datasets/cbdb/cbdb-sqlite/resolve/main/latest.7z?download=true"
r = requests.get(url, verify=False)
os.makedirs("./cbdb_sqlite", exist_ok=True)
with open("./cbdb_sqlite/latest.7z", "wb") as f:
    f.write(r.content)

d:\anaconda\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'hf-mirror.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
d:\anaconda\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cdn-lfs-us-1.hf-mirror.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [2]:
# import glob
import os

latest_file = "./cbdb_sqlite/latest.7z"
latest_db = latest_file.split("/")[2].split(".")[0] + ".db"
print(latest_db)
if os.path.isfile(latest_db):
    os.remove(latest_db)

latest.db


In [3]:
# Explode latest database file
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive(latest_file).extractall(".")

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [4]:
# Create connection to database
import sqlite3
import pandas as pd
global CONN
database_file = os.path.basename(latest_file).split(".")[0]+".db"
CONN = sqlite3.connect(database_file)

In [5]:
sql_str = "SELECT bm.c_dy, d.c_dynasty_chn, count(distinct m.c_textid) as count\
        FROM biog_main bm\
        JOIN (\
            SELECT bs.c_personid, bs.c_textid\
            FROM biog_source_data bs\
        ) m ON bm.c_personid = m.c_personid\
        JOIN dynasties d ON bm.c_dy = d.c_dy\
        GROUP BY bm.c_dy;"
dycount_df = pd.read_sql_query(sql_str, CONN).drop_duplicates().dropna()
dycount_df

,c_dy,c_dynasty_chn,count
0,0,未詳,114
1,1,漢前,4
2,2,秦漢,17
3,3,三國,2
4,4,南北朝,8
...,...,...,...
67,76,西燕,1
68,77,周,12
69,78,西夏,2
70,81,鄭（王世充）,1


In [6]:
sql_str="select bm.c_dy, d.c_dynasty, d.c_dynasty_chn, group_concat(bs.c_textid)as bookids\
        from biog_main bm\
        join biog_source_data bs on bm.c_personid = bs.c_personid\
        join dynasties d on bm.c_dy = d.c_dy\
        group by bm.c_dy;"
dycount_df = pd.read_sql_query(sql_str, CONN).drop_duplicates().dropna()
dycount_df['bookids'] = dycount_df['bookids'].apply(lambda x: ','.join(set(x.split(','))) if isinstance(x, str) else x)  
dycount_df['count'] = dycount_df['bookids'].apply(lambda x: len(x.split(',')) if isinstance(x, str) else 0)  
dycount_df

,c_dy,c_dynasty,c_dynasty_chn,bookids,count
0,0,unknown,未詳,"32039,2066,38733,2152,18536,38630,11660,2229,1...",114
1,1,Pre-Han,漢前,"5908,6134,9601,7596",4
2,2,QinHan,秦漢,"6210,17138,3689,38651,9602,38699,38742,38772,5...",17
3,3,SanGuo,三國,"9602,9601",2
4,4,NanBei Chao,南北朝,"4801,32039,32038,2067,38295,32033,9601,5906",8
...,...,...,...,...,...
67,76,Western Yan (16 states),西燕,24309,1
68,77,Zhou (Wu Zetian),周,"4801,32039,40303,32038,3689,4578,38295,38296,2...",12
69,78,Western Xia,西夏,"7596,17138",2
70,81,Zheng (Wang Shichong),鄭（王世充）,32038,1


In [7]:
dycount_df_filtered = dycount_df[dycount_df["c_dy"].isin([6, 15, 16, 17, 18, 19 ,20])]  
dycount_df_filtered

,c_dy,c_dynasty,c_dynasty_chn,bookids,count
6,6,Tang,唐,"32039,36219,38656,20920,38861,64948,38733,4515...",120
15,15,Song,宋,"38656,64948,2508,38733,38630,38758,58265,5882,...",374
16,16,Liao,遼,"40303,7596,17138,8947,24309,32033,9601,6267",8
17,17,Jin,金,"27858,38651,5724,38802,5838,38733,6134,38457,3...",39
18,18,Yuan,元,"36219,2066,38308,38656,38313,38733,38322,4515,...",156
19,19,Ming,明,"38656,18174,38733,32092,38496,18536,38758,3877...",487
20,20,Qing,清,"36219,2066,32077,38656,38832,38861,32000,20647...",213


In [8]:
dycount_df_filtered.to_csv('text_statistic.csv', index=False)

In [9]:
# Read the content of old JavaScript files
with open('script.js', 'r') as file:  
    js_code = file.read()  
  
# Delete old data  
start_delimiter = 'const chart3Data = ['  
end_delimiter = '];// Chart 3 Data'
start_index = js_code.find(start_delimiter)  
end_index = js_code.find(end_delimiter) + len(end_delimiter)  
js_code = js_code[:start_index] + js_code[end_index:]
# new data
js_data = ',\n'.join(    
    f'{{ label: "{row["c_dynasty"]}", value: {row["count"]}, id: "{row["c_dynasty"]}"}}'    
    for index, row in dycount_df_filtered.iterrows()    
)  
new_data = f'{start_delimiter}\n{js_data}\n{end_delimiter}'  
  
js_code = new_data +'\n' + js_code  
  
# Write updated JavaScript file 
with open('script.js', 'w') as file:  
    file.write(js_code)   

In [10]:
# update the date
import datetime  
import re  

svg_path = os.path.join(os.getcwd(), 'images', 'svg6.svg')  
with open(svg_path, 'r') as file:  
    svg_content = file.read()  
  
# get current date  
current_time = datetime.datetime.now().strftime('%Y年%m月%d日') 
current_time_english = datetime.datetime.now().strftime('%B %d, %Y') 
  
updated_svg_content = re.sub(r'更新日期：\d+年\d+月\d+日', f'更新日期：{current_time}', svg_content)    
updated_svg_content = re.sub(r'Updated on: \w+ \d+, \d+', f'Updated on: {current_time_english}', updated_svg_content)
with open(svg_path, 'w') as file:  
    file.write(updated_svg_content) 

In [11]:
# close the db
CONN.close()